In [1]:
import gym
import pandas as pd
from collections import defaultdict
import random


In [2]:
env = gym.make('Blackjack-v0')

In [4]:
env.observation_space

Tuple(Discrete(32), Discrete(11), Discrete(2))

In [5]:
env.action_space

Discrete(2)

> Initializing the dicts for storing Q values, total return, and no of times of state-action pairs

In [6]:
Q = defaultdict(float)
total_return = defaultdict(float)
N = defaultdict(int)

> Defining the epsilon-greedy policy

In [7]:
def epsilon_greedy_policy(state, Q):
  epsilon = 0.5

  if random.uniform(0,1) < epsilon:
    return env.action_space.sample()

  else:
    return max(list(range(env.action_space.n)), key= lambda x: Q[(state, x)])
    

> Generating an episode

In [8]:
num_timesteps = 100
def generate_episode(Q):

  episode = []
  state = env.reset()

  for t in range(num_timesteps):

    action = epsilon_greedy_policy(state, Q)

    next_state, reward, done, info = env.step(action)
    episode.append((state, action, reward))

    if done:
      break

    state = next_state

  return episode

> Computing Optimal Policy

In [9]:
Q

defaultdict(float, {})

In [13]:
# testing
epsilon_greedy_policy(env.reset(), Q)


0

In [14]:
num_iterations = 50000

for i in range(num_iterations):

  episode = generate_episode(Q)

  all_state_action_pairs = [(s,a) for (s,a,r) in episode]

  rewards = [r for (s,a,r) in episode]

  for t, (state, action, reward) in enumerate(episode):

    if not (state, action) in all_state_action_pairs[0:t]:
      R = sum(rewards[t:])
      total_return[(state, action)] = total_return[(state, action)] + R
      N[(state, action)] += 1

      # Computing the Q value by just taking the average
      Q[(state, action)] = total_return[(state, action)] /N[(state, action)]

In [15]:
df = pd.DataFrame(Q.items(),columns=['state_action pair','value'])


In [16]:
df.head(5)


,state_action pair,value
0,"((18, 3, False), 0)",0.131313
1,"((18, 3, False), 1)",-0.779221
2,"((21, 10, True), 0)",0.858015
3,"((21, 10, True), 1)",-0.064655
4,"((17, 7, False), 0)",-0.050909


**Till now, we have covered the MC method, and how it addresses the limitations of Bellman Equation. But there are some limitations for MC Method as well:**

> it is applicable only to episodic tasks. We learned that in the Monte Carlo method, we compute the value of the state by taking the average return of the state and the return is the sum of rewards of the episode. But when there is no episode, that is, if our task is a continuous task (non-episodic task), then we cannot apply the Monte Carlo method.